# Задание 1

    Напишите функции для сортировки пузырьком, выбором и вставкой без использования OpenMP.

In [ ]:
%%writefile task1.cpp

#include <iostream>
#include <chrono>
#include <cstdlib>

using namespace std;
using namespace chrono;

// Сортировка пузырьком
void bubbleSort(int arr[], int n) {
    for (int i = 0; i < n - 1; i++) {
        for (int j = 0; j < n - i - 1; j++) {
            if (arr[j] > arr[j + 1]) {
                swap(arr[j], arr[j + 1]);
            }
        }
    }
}

// Сортировка выбором
void selectionSort(int arr[], int n) {
    for (int i = 0; i < n - 1; i++) {
        int minIndex = i;

        for (int j = i + 1; j < n; j++) {
            if (arr[j] < arr[minIndex]) {
                minIndex = j;
            }
        }

        swap(arr[i], arr[minIndex]);
    }
}

// Сортировка вставками
void insertionSort(int arr[], int n) {
    for (int i = 1; i < n; i++) {
        int key = arr[i];
        int j = i - 1;

        while (j >= 0 && arr[j] > key) {
            arr[j + 1] = arr[j];
            j--;
        }

        arr[j + 1] = key;
    }
}

// Копирование массива
void copyArray(int source[], int dest[], int n) {
    for (int i = 0; i < n; i++) {
        dest[i] = source[i];
    }
}

// main
int main() {
    const int N = 1000;   // можно менять размер массива
    int original[N];

    // Заполнение массива случайными числами
    srand(time(nullptr));
    for (int i = 0; i < N; i++) {
        original[i] = rand() % 10000;
    }

    int arr1[N], arr2[N], arr3[N];

    // ---------- Bubble Sort ----------
    copyArray(original, arr1, N);
    auto start_bubble = high_resolution_clock::now();
    bubbleSort(arr1, N);
    auto end_bubble = high_resolution_clock::now();
    double time_bubble =
        duration<double, milli>(end_bubble - start_bubble).count();

    // ---------- Selection Sort ----------
    copyArray(original, arr2, N);
    auto start_selection = high_resolution_clock::now();
    selectionSort(arr2, N);
    auto end_selection = high_resolution_clock::now();
    double time_selection =
        duration<double, milli>(end_selection - start_selection).count();

    // ---------- Insertion Sort ----------
    copyArray(original, arr3, N);
    auto start_insertion = high_resolution_clock::now();
    insertionSort(arr3, N);
    auto end_insertion = high_resolution_clock::now();
    double time_insertion =
        duration<double, milli>(end_insertion - start_insertion).count();

    // ---------- Результаты ----------
    cout << "Размер массива: " << N << endl;
    cout << "Сортировка пузырьком: " << time_bubble << " мс" << endl;
    cout << "Сортировка выбором:   " << time_selection << " мс" << endl;
    cout << "Сортировка вставками: " << time_insertion << " мс" << endl;

    return 0;
}


Overwriting task1.cpp


In [10]:
! g++ -std=c++11 task1.cpp -o task1
! ./task1

Размер массива: 1000
Сортировка пузырьком: 2.12021 мс
Сортировка выбором:   1.17088 мс
Сортировка вставками: 0.586667 мс


# Задание 2

    Используйте директивы OpenMP для распараллеливания внешних циклов. 
    Протестируйте производительность каждой сортировки на массивах разного размера (например, 1000, 10,000 и 100,000 элементов).

In [12]:
%%writefile task2.cpp

#include <iostream>
#include <vector>
#include <cstdlib>
#include <ctime>
#include <chrono>
#include <omp.h>

using namespace std;
using namespace chrono;

// Пузырьковая сортировка (OpenMP)
void bubbleSortOMP(vector<int>& arr) {
    int n = arr.size();

    for (int i = 0; i < n - 1; i++) {
        #pragma omp parallel for
        for (int j = 0; j < n - i - 1; j++) {
            if (arr[j] > arr[j + 1]) {
                swap(arr[j], arr[j + 1]);
            }
        }
    }
}

// Сортировка выбором (OpenMP)
void selectionSortOMP(vector<int>& arr) {
    int n = arr.size();

    for (int i = 0; i < n - 1; i++) {
        int min_idx = i;

        #pragma omp parallel for
        for (int j = i + 1; j < n; j++) {
            #pragma omp critical
            {
                if (arr[j] < arr[min_idx]) {
                    min_idx = j;
                }
            }
        }
        swap(arr[i], arr[min_idx]);
    }
}

// Сортировка вставками (OpenMP)
void insertionSortOMP(vector<int>& arr) {
    int n = arr.size();

    for (int i = 1; i < n; i++) {
        int key = arr[i];
        int j = i - 1;

        while (j >= 0 && arr[j] > key) {
            arr[j + 1] = arr[j];
            j--;
        }
        arr[j + 1] = key;
    }
}

// Генерация массива
vector<int> generateArray(int size) {
    vector<int> arr(size);
    for (int i = 0; i < size; i++) {
        arr[i] = rand() % 100000;
    }
    return arr;
}

// Тестирование сортировок
void testSorts(int size) {
    cout << "\nРазмер массива: " << size << endl;

    vector<int> arr1 = generateArray(size);
    vector<int> arr2 = arr1;
    vector<int> arr3 = arr1;

    auto start = high_resolution_clock::now();
    bubbleSortOMP(arr1);
    auto end = high_resolution_clock::now();
    cout << "Пузырьковая сортировка (OpenMP): "
         << duration<double, milli>(end - start).count() << " мс" << endl;

    start = high_resolution_clock::now();
    selectionSortOMP(arr2);
    end = high_resolution_clock::now();
    cout << "Сортировка выбором (OpenMP):   "
         << duration<double, milli>(end - start).count() << " мс" << endl;

    start = high_resolution_clock::now();
    insertionSortOMP(arr3);
    end = high_resolution_clock::now();
    cout << "Сортировка вставками (OpenMP): "
         << duration<double, milli>(end - start).count() << " мс" << endl;
}

// main
int main() {
    srand(time(nullptr));

    testSorts(1000);
    testSorts(10000);
    testSorts(100000);

    return 0;
}

Writing task2.cpp


In [16]:
!g++-15 -fopenmp task2.cpp -o task2
!./task2



Размер массива: 1000
Пузырьковая сортировка (OpenMP): 53.528 мс
Сортировка выбором (OpenMP):   42.86 мс
Сортировка вставками (OpenMP): 1.693 мс

Размер массива: 10000
Пузырьковая сортировка (OpenMP): 428.199 мс
Сортировка выбором (OpenMP):   1499.19 мс
Сортировка вставками (OpenMP): 173.684 мс

Размер массива: 100000
Пузырьковая сортировка (OpenMP): 15831 мс
Сортировка выбором (OpenMP):   133716 мс
Сортировка вставками (OpenMP): 17516.5 мс


# Задание 3

    Сравнение производительности:
    Измерьте время выполнения последовательных и параллельных версий каждой сортировки, используя библиотеку <chrono>.
    Сравните результаты и сделайте выводы.

## Сравнение производительности последовательных и параллельных сортировок (OpenMP)

### Исходные данные измерений

**Размер массива: 1000**
   - Пузырьковая сортировка (OpenMP): **53.528 мс**
   - Сортировка выбором (OpenMP): **42.86 мс**
   - Сортировка вставками (OpenMP): **1.693 мс**

   **Размер массива: 10 000**
   - Пузырьковая сортировка (OpenMP): **428.199 мс**
   - Сортировка выбором (OpenMP): **1499.19 мс**
   - Сортировка вставками (OpenMP): **173.684 мс**

   **Размер массива: 100 000**
   - Пузырьковая сортировка (OpenMP): **15 831 мс**
   - Сортировка выбором (OpenMP): **133 716 мс**
   - Сортировка вставками (OpenMP): **17 516.5 мс**

---

### Анализ результатов

#### Сортировка вставками (Insertion Sort)

- Является **самой быстрой** реализацией для всех размеров массивов.
- При размере 1000 элементов демонстрирует **минимальное время выполнения**.
- Даже при 100 000 элементов остаётся существенно быстрее сортировки выбором.

**Причины эффективности:**
- меньшее количество обменов элементов;
- лучшее использование кэш-памяти;
- устойчивость к частично отсортированным данным;
- более удачное распараллеливание внешнего цикла.

---

#### Пузырьковая сортировка (Bubble Sort)

- Демонстрирует **стабильную, но низкую производительность**.
- Время выполнения растёт квадратично при увеличении размера массива.
- Для массивов большого размера алгоритм становится **непрактичным**, даже при использовании OpenMP.

---

#### Сортировка выбором (Selection Sort)

- Показала **наихудшие результаты** при средних и больших размерах данных.
- На массиве из 100 000 элементов время выполнения превышает **130 секунд**.

**Причины низкой эффективности:**
- большое количество сравнений;
- жёсткие зависимости данных;
- высокая стоимость синхронизации между потоками.

---

### Сравнительная таблица времени выполнения

| Размер массива | Insertion Sort | Bubble Sort | Selection Sort |
|---------------|---------------|------------|---------------|
| 1 000         | 1.7 мс        | 53.5 мс    | 42.9 мс       |
| 10 000        | 173.7 мс      | 428.2 мс   | 1499.2 мс     |
| 100 000       | 17.5 с        | 15.8 с     | 133.7 с       |

---

### Выводы

1. Все рассмотренные алгоритмы имеют квадратичную сложность **O(N²)**, что подтверждается ростом времени выполнения при увеличении объёма данных.
2. Использование OpenMP **не устраняет алгоритмическую неэффективность** простых сортировок.
3. Среди параллельных реализаций:
   - **Insertion Sort** показала наилучшие результаты;
   - **Selection Sort** оказалась наименее подходящей для параллелизации.
4. Для получения существенного ускорения необходимо использовать алгоритмы сортировки с лучшей асимптотикой и адаптированные под параллельные вычисления.
